# Exemplos de uso do mongoDB e do SQLite

## Carregando variáveis de ambiente

In [1]:
from src.settings import BASE_DIR, SQLITE_DB_NAME, MONGO_URI, MONGO_DB_NAME, MONGO_COLLECTION

## Criando base de dados relacional com SQLite

### Conectando-se ao SQLite
Primeiro, vamos importar a biblioteca sqlite3 e estabelecer uma conexão com o banco de dados SQLite. Se o arquivo do banco de dados não existir, ele será criado automaticamente.

In [ ]:
import sqlite3

# Conectar ao banco de dados SQLite (será criado se não existir)
db_path = f"{BASE_DIR}/data/sqlite/{SQLITE_DB_NAME}"
connection = sqlite3.connect(db_path)
cursor = connection.cursor()

### Criando Tabelas
Agora, vamos criar três tabelas: MOVIES, MOVIES_CUSTOMER e WATCHED_MOVIE

In [2]:
# Criar tabela MOVIES
cursor.execute("""
    CREATE TABLE IF NOT EXISTS MOVIES (
        MOVIE_ID INTEGER PRIMARY KEY,
        TITLE TEXT,
        GENRES TEXT,
        SUMMARY TEXT
    )
""")

# Criar tabela MOVIES_CUSTOMER
cursor.execute("""
    CREATE TABLE IF NOT EXISTS MOVIES_CUSTOMER (
        CUST_ID INTEGER PRIMARY KEY,
        FIRSTNAME TEXT,
        LASTNAME TEXT
    )
""")

# Criar tabela WATCHED_MOVIE
cursor.execute("""
    CREATE TABLE IF NOT EXISTS WATCHED_MOVIE (
        DAY_ID TEXT,
        MOVIE_ID INTEGER,
        PROMO_CUST_ID INTEGER,
        PRIMARY KEY (DAY_ID, MOVIE_ID, PROMO_CUST_ID),
        FOREIGN KEY (MOVIE_ID) REFERENCES MOVIES(MOVIE_ID),
        FOREIGN KEY (PROMO_CUST_ID) REFERENCES MOVIES_CUSTOMER(CUST_ID)
    )
""")

### Inserindo os dados nas tabelas

In [3]:
# Inserir dados na tabela MOVIES
cursor.executemany("""
    INSERT INTO MOVIES (MOVIE_ID, TITLE, GENRES, SUMMARY) VALUES (?, ?, ?, ?)
""", [
    (1, 'Inception', '{"Action": "Sci-Fi"}', 'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O.'),
    (2, 'The Matrix', '{"Action": "Sci-Fi"}', 'A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.'),
    (3, 'The Godfather', '{"Drama": "Crime"}', 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.'),
    (4, 'Titanic', '{"Romance": "Drama"}', 'A seventeen-year-old aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic.'),
    (5, 'Toy Story', '{"Animation": "Adventure"}', 'A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy''s room.')
])

# Inserir dados na tabela MOVIES_CUSTOMER
cursor.executemany("""
    INSERT INTO MOVIES_CUSTOMER (CUST_ID, FIRSTNAME, LASTNAME) VALUES (?, ?, ?)
""", [
    (101, 'John', 'Doe'),
    (102, 'Jane', 'Smith'),
    (103, 'Sam', 'Wilson'),
    (104, 'Emily', 'Clark'),
    (105, 'Michael', 'Johnson')
])

# Inserir dados na tabela WATCHED_MOVIE
cursor.executemany("""
    INSERT INTO WATCHED_MOVIE (DAY_ID, MOVIE_ID, PROMO_CUST_ID) VALUES (?, ?, ?)
""", [
    ('2024-10-30 12:34:56', 1, 101),
    ('2024-10-31 12:34:56', 2, 101),
    ('2024-09-30 12:34:56', 3, 101),
    ('2024-10-31 09:15:23', 2, 102),
    ('2024-11-01 16:45:12', 3, 103),
    ('2024-11-02 18:22:43', 4, 104),
    ('2024-11-03 20:01:00', 5, 105)
])

# Confirmar as alterações
connection.commit()

### Consultando os Dados
Para consultar os dados inseridos e verificar as informações, podemos executar uma simples consulta SQL:

In [4]:
cursor.execute("""
    SELECT c.FIRSTNAME, c.LASTNAME, m.TITLE, w.DAY_ID
    FROM MOVIES_CUSTOMER c
    JOIN WATCHED_MOVIE w ON c.CUST_ID = w.PROMO_CUST_ID
    JOIN MOVIES m ON w.MOVIE_ID = m.MOVIE_ID
""")

for row in cursor.fetchall():
    print(row)


('John', 'Doe', 'Inception', '2024-10-30 12:34:56')
('John', 'Doe', 'The Matrix', '2024-10-31 12:34:56')
('John', 'Doe', 'The Godfather', '2024-09-30 12:34:56')
('Jane', 'Smith', 'The Matrix', '2024-10-31 09:15:23')
('Sam', 'Wilson', 'The Godfather', '2024-11-01 16:45:12')
('Emily', 'Clark', 'Titanic', '2024-11-02 18:22:43')
('Michael', 'Johnson', 'Toy Story', '2024-11-03 20:01:00')


## Criando exemplos de contexto par ao agente no MongoDB

### Estabelecendo conexão com o mongodb

In [9]:
from pymongo import MongoClient

client = MongoClient(MONGO_URI)
client.drop_database(MONGO_DB_NAME)
db = client[MONGO_DB_NAME]

### Criando coleções no MongoDB

In [33]:
# Coleções
nodes = db["nodes"]
edges = db["edges"]

### Carregando dados dos jsons

In [ ]:
import json

# Função utilitária para carregar JSON
def load_json(path):
    with open(path, "r") as f:
        return json.load(f)

# Carregar dados
file_names = ["customers", "movies", "watched"]
destinations = {}
base_path = "data/{file_name}.json"
for file_name in file_names:
    destinations[file_name] = load_json(base_path.format(file_name=file_name))

customers = destinations['customers']
movies = destinations['movies']
watched = destinations['watched']

In [38]:
customers

[{'CUST_ID': 101, 'FIRSTNAME': 'John', 'LASTNAME': 'Doe'}]

### Inserindo dados nos nós

In [20]:
def insert_nodes(data, type, key_id, collection):
    documents = []
    for item in data:
        doc = {
            "_id": f"{type.lower()}_{item[key_id]}",
            "type": type,
            "properties": item
        }
        documents.append(doc)

    if documents:
        collection.insert_many(documents)
        print(f"{len(documents)} nós do tipo '{type}' inseridos com sucesso.")
    else:
        print(f"Nenhum dado para inserir como '{type}'.")

In [ ]:
insert_nodes(customers, "CUSTOMER", "CUST_ID", nodes)
insert_nodes(movies, "MOVIE", "MOVIE_ID", nodes)

1 nós do tipo 'CUSTOMER' inseridos com sucesso.
1 nós do tipo 'MOVIE' inseridos com sucesso.


### Inserindo dados nos vértices

In [22]:
def insert_edges(data, type, fn_source, fn_destination, collection):
    documents = []
    for item in data:
        edge_id = f"{type.lower()}_{item['DAY_ID'].replace(' ', '_')}_{item['MOVIE_ID']}_{item['PROMO_CUST_ID']}"
        doc = {
            "_id": edge_id,
            "type": type,
            "source": fn_source(item),
            "destination": fn_destination(item),
            "properties": item
        }
        documents.append(doc)

    if documents:
        collection.insert_many(documents)
        print(f"{len(documents)} edges do tipo '{type}' inseridos com sucesso.")
    else:
        print(f"Nenhum edge do tipo '{type}' para inserir.")

In [ ]:
# insert_edges(
#     watched,
#     tipo="WATCHED",
#     fn_source=lambda d: f"cust_{d['PROMO_CUST_ID']}",
#     fn_destination=lambda d: f"movie_{d['MOVIE_ID']}",
#     collection=edges
# )

In [23]:
# Inserir EDGE: WATCHED
edges.insert_one({
    "_id": "watch_2024-10-30_1_101",
    "type": "WATCHED",
    "source": "cust_101",           # refers to CUSTOMER
    "destination": "movie_1",       # refers to MOVIE
    "properties": {
        "DAY_ID": "2024-10-30 12:34:56",
        "MOVIE_ID": 1,
        "PROMO_CUST_ID": 101
    }
})


InsertOneResult('watch_2024-10-30_1_101', acknowledged=True)